In [35]:
import os, sys

cwd = os.getcwd()
module_path = os.path.abspath(os.path.join('..'))

sys.path.insert(0, os.path.join(cwd, "../"))
sys.path.insert(0, os.path.join(cwd, "../student_life/"))
sys.path.insert(0, os.path.join(cwd, "../student_life/src/"))
sys.path.insert(0, os.path.join(cwd, "../student_life/src/bin/"))


import copy
from scipy import stats
import pickle
import importlib
import src.utils.student_utils as student_utils
import src.utils.data_conversion_utils as conversion_utils
from collections import defaultdict
import geopy.distance
from sklearn.metrics.pairwise import euclidean_distances

import src.definitions as definitions
import pandas as pd
import numpy as np
from IPython.display import display
from src.data_manager import student_life_var_binned_data_manager
importlib.reload(student_life_var_binned_data_manager)
importlib.reload(student_utils)

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline  

print(module_path)

Student ID couldn't be converted to Integer!
/Users/nsimsiri/Documents/code/ml/MultiRes/student_life


In [2]:
import geopy.distance as euclidean_distances
COORD_DIST = lambda p1, p2: euclidean_distances.distance(p1, p2).km
from scipy.spatial import ConvexHull
from area import area
from editdistance import eval as edit_distance
from sklearn.cluster import MeanShift, estimate_bandwidth
import scipy
from datetime import datetime, timedelta
np.seterr(divide='ignore', invalid='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [3]:
keyword = 'gps'
def load_features(student_id=2):
    path = "{}/data/student_life_minimal_processed_data/student_{}".format(module_path, student_id)
    assert(os.path.exists(path))
    feats = {}
    for _file in os.listdir(path):
        if keyword not in _file and 'stress' not in _file: 
            continue
        feat_path = path + "/" + _file
        df = pd.read_csv(feat_path)
        print('student {} loaded feat: {}'.format(student_id, _file))      
        feat_name = _file.replace('.csv', '')
        feats[feat_name] = df
    return feats
        
    
feats = load_features(student_id=46)
print(len(feats))

student 46 loaded feat: stress_details.csv
student 46 loaded feat: gps_details.csv
2


Location Features (8 total):
 - total distance covered in a day
 - maximum 2-point displacement in a day
 - distance standard deviatio
 - number of different areas visited by tiles approximation
 - total spatial coverage by convex hull,
 - difference in sequence of tiles covered compared to previous day
 - difference in sequence of clusters visited compared to previous day
 - distance entropy

4 Temporal one-hot features 
- weekends, start of term, mid-term, end of term

https://www.ucl.ac.uk/~ucfamus/papers/digitalbiomarkers17.pdf

In [4]:
def index_time_to_df(df):
    df.loc[:, 'time'] = pd.to_datetime(df.loc[:,'time'])
    df = df.set_index('time')
    return df
location = feats['gps_details']
location = index_time_to_df(location)

### Stress

In [5]:
def s_filter(pd, agg_fn, date_range):
    lo, hi = date_range
    tmp = pd.loc[lo:hi]
    out = agg_fn(tmp)
    if out is float('NaN'):
        return 0.0
    return out
    
def get_stress_labels_with_time(feats):
    stress = feats['stress_details'].copy()
    stress = stress.drop_duplicates(subset='time', keep='last')
    N = len(stress)
    stress_median = stress['stress_level'].median()
    stress = index_time_to_df(stress)
    stress['stress'] = [0.0 for i in range(N)]
    stress['range'] = [None for i in range(N)]
    def bin_stress_to_classes(stress_val):
        if stress_val < stress_median: 
            return 0
        elif stress_val > stress_median:
            return 2
        else:
            return 1
        return bin_stress_to_classes

    for idx in stress.index:
        raw_stress_val = stress.loc[idx]['stress_level']
        stress.at[idx,'stress'] = bin_stress_to_classes(raw_stress_val)
        stress.at[idx, 'range'] = [idx-pd.Timedelta(days=1), idx]
    return stress
# stress_labels = get_stress_labels_with_time(feats)
# display(stress_labels[:3])
# a,b = stress_labels.iloc[4]['range']
# print(a,b)
# display(stress_labels[a-pd.Timedelta(days=1):a+pd.Timedelta(days=1)])

### (1, 2) Total Distance Covered, Max Displacement 

In [6]:
def to_distance(_location):
    location = _location.copy()
    location['distance'] = [0 for x in range(len(location))]

    for i in range(1, len(location)):
        row_cur = location.iloc[i]
        row_prev = location.iloc[i-1]
        p1 = (row_cur['latitude'], row_cur['longitude'])
        p2 = (row_prev['latitude'], row_prev['longitude'])
        dist = euclidean_distances.distance(p1, p2).km
        location.iloc[i, location.columns.get_loc('distance')]= dist
    return location

def get_feat_distance_covered(distance, stress_labels):
    # display(conversation)
    out = stress_labels.copy()
    N = len(out)
    out['distance'] = [0.0 for i in range(N)]
    out['displacement'] = [0.0 for i in range(N)]
    
    def _sum(subset):
        return subset.sum()
    def _max(subset):
        return subset.max()
    
    for idx in out.index:
        date_range = out.at[idx,'range']
        dist_feats = distance['distance']
        out.at[idx, 'distance'] = s_filter(dist_feats, _sum, date_range)
        out.at[idx, 'displacement'] = s_filter(dist_feats, _max, date_range)
        out = out.fillna(0.0)
    
    return out

# distance = to_distance(location)
# next_stress_labels = get_feat_distance_covered(distance, stress_labels)
# display(next_stress_labels[:10])

### (3) Standard Deviation

In [7]:
def get_std_distance(distance, stress_labels):
    out = stress_labels.copy()
    N = len(out)
    out['distance_std'] = [0.0 for i in range(N)]
    
    def _std(subset):
        return subset.std()
    
    for idx in out.index:
        date_range = out.at[idx,'range']
        dist_feats = distance['distance']
        out.at[idx, 'distance_std'] = s_filter(dist_feats, _std, date_range)
        out = out.fillna(0.0)
    return out

# next_stress_labels_2 = get_std_distance(distance, next_stress_labels)
# display(next_stress_labels_2[:10])

### (4) Spatial Tile - unique tile visited

In [8]:

def build_spatial_map(_location, bound=50):
    location = _location.copy()
    lats  = location['latitude']
    longs = location['longitude'] 
    min_lat, max_lat = lats.min(), lats.max()
    min_long, max_long = longs.min(), longs.max()
    TL = [min_lat, max_long]
    TR = [max_lat, max_long]
    BL = [min_lat, min_long]
    BR = [max_lat, min_long]

    location['area'] = ["" for x in range(len(location))]
    for i in range(len(location)):
        row = location.iloc[i]
        lat, long = row['latitude'], row['longitude']
        x_dist = COORD_DIST([lat, long], [min_lat, long])
        y_dist = COORD_DIST([lat, long], [lat, min_long])
        hash_x, hash_y = int(x_dist/bound), int(y_dist/bound)
        hash_xy = "{}-{}".format(hash_x, hash_y)
        location.iloc[i, location.columns.get_loc('area')] = hash_xy
    return pd.DataFrame(location['area'])

def bin_areas_by_day(areas, stress_labels):
    def bin_day(subset):
        tiles = [x[0] for x in subset.values]
        return tiles
    
    out = stress_labels.copy()
    N = len(out)
    out['area'] = [[] for i in range(N)]
    
    for idx in out.index:
        date_range = out.at[idx,'range']
        out.at[idx, 'area'] = s_filter(areas, bin_day, date_range)
    return out

# areas = build_spatial_map(location) # time -> tile
# # display(areas[:6].values)
# next_stress_labels_3 = bin_areas_by_day(areas, next_stress_labels_2)
# display(area_days[:6])

In [9]:
def unique_tile_feats(stress_labels):
    out = stress_labels.copy()
    N = len(stress_labels)
    out['unique_tile_count'] = [0.0 for x in range(N)]
    for i in range(N):
        tile_set = set(out.iloc[i, out.columns.get_loc('area')])
        out.iloc[i, out.columns.get_loc('unique_tile_count')] = len(tile_set)
    return out
        
# next_stress_labels_4 = unique_tile_feats(next_stress_labels_3)
# display(next_stress_labels_4[:10])


### (5) Convex Hull

In [10]:
def convex_hull_feats(locations, stress_labels):
    def bin_convex(subset):
        coords = subset[['latitude','longitude']].values
        if(len(coords)<=2):
            return 0
        hull = ConvexHull(coords)
        points = [list(hull.points[ix]) for ix in hull.vertices]
        obj = {'type':'Polygon','coordinates':[points]}
        area_result = area(obj)/100
        return area_result

    N = len(stress_labels)
    out = stress_labels.copy()
    out['convex_hull_area'] = [[] for i in range(N)]
    
    for idx in out.index:
        date_range = out.at[idx,'range']
        out.at[idx, 'convex_hull_area'] = s_filter(locations, bin_convex, date_range)
    return out
    

# next_stress_labels_5 = convex_hull_feats(location, next_stress_labels_4)
# display(next_stress_labels_5[:3])
# display(next_stress_labels_5['convex_hull_area'].values)


### (6) difference in sequence of tiles covered compared to previous day

In [11]:
def prev_edit_dist_feats(stress_labels):
    N = len(stress_labels)
    out = stress_labels.copy()
    out['tile_change'] = [0.0 for i in range(N)]
    for i in range(1, N):
        cur_tiles  = out.iloc[i, out.columns.get_loc('area')]
        prev_tiles = out.iloc[i-1, out.columns.get_loc('area')]
        change = edit_distance(cur_tiles, prev_tiles)
        out.iloc[i, out.columns.get_loc('tile_change')] = change
    return out

# next_stress_labels_6 = prev_edit_dist_feats(next_stress_labels_5)
# display(next_stress_labels_6[:10])

### (7) difference in sequence of clusters visited compared to previous day

In [12]:
def location_cluster_feats(locations, stress_labels):
    def bin_location_cluster(subset):
        X = np.array(subset['cluster'].values.tolist())
        if (len(X)<=0): 
            return 0
        ms = MeanShift()
        ms.fit(X)
        c = len(ms.cluster_centers_)
        return c
    
    tmp = locations.copy()
    tmp['cluster'] = locations[['latitude','longitude']].values.tolist()
    
    N = len(stress_labels)
    out = stress_labels.copy()
    out['cluster'] = [0.0 for i in range(N)]
    
    for idx in out.index:
        date_range = out.at[idx,'range']
        out.at[idx, 'cluster'] = s_filter(tmp, bin_location_cluster, date_range)
    return out
    
# next_stress_labels_7 = location_cluster_feats(location, next_stress_labels_6)
# display(next_stress_labels_7[:3])
# display(next_stress_labels_7['cluster'].values)


### (8) Shannon Entropy every 10 minutes per day

In [13]:
def entropy_feats(distance, stress_labels):
    def bin_shannon(subset):
        x = subset['distance'].values
        entropy = scipy.stats.entropy(x)
        return entropy
    
    tmp = pd.DataFrame(distance['distance'].copy())
    tmp = tmp.resample("10T").sum()
    tmp = tmp.fillna(0.0)
    
    N = len(stress_labels)
    out = stress_labels.copy()
    out['entropy'] = [0.0 for i in range(N)]
    
    for idx in out.index:
        date_range = out.at[idx,'range']
        out.at[idx, 'entropy'] = s_filter(tmp, bin_shannon, date_range)
        
    return out
    
# display(distance)
# next_stress_levels_8 = entropy_feats(distance, next_stress_labels_7)
# display(next_stress_levels_8[:6])

### (9-12) Temporal Features

In [14]:
TO_YEAR = lambda x: x.astype('datetime64[Y]').astype(int) + 1970
TO_MONTH = lambda x: x.astype('datetime64[M]').astype(int)  % 12 + 1
TO_DAY = lambda x: x.astype('datetime64[D]').astype(int)
def to_year_month_date(x):
    year, month, day = str(x).split('T')[0].split("-")
    year, month, day = int(year), int(month), int(day)
    return year, month, day

def is_weekend(dt64):
    y,m,d = to_year_month_date(dt64)
    x = datetime(y,m,d,0,9)
    return x.isoweekday() in [6,7]

def temporal_feats(stress_levels):
    N = len(stress_levels)
    out = stress_levels.copy()
    mid_start_raw = '2013-04-15T00:00:00.000000000'#Monday, April 15, 2013
    mid_end_raw = '2013-05-06T00:00:00.000000000'# Monday, May 6, 2013
    mid_start = np.datetime64(mid_start_raw)
    mid_end   = np.datetime64(mid_end_raw)
    out['start_term'] = [0.0 for i in range(N)]
    out['end_term'] = [0.0 for i in range(N)]
    out['mid_term'] = [0.0 for i in range(N)]
    out['weekends'] = [0.0 for i in range(N)]
    for idx in out.index.values:
        weekend_col = out.columns.get_loc('weekends')
        out.at[idx,'weekends'] = int(is_weekend(idx))
        x_term_col = ""
        if (idx <= mid_start):
            x_term_col = 'start_term'
        elif (idx >= mid_end):
            x_term_col = 'end_term'
        else:
            x_term_col = 'mid_term'
        out.at[idx, x_term_col] = 1
    return out
        
        
# next_stress_level_9 = temporal_feats(next_stress_levels_8)
# display(next_stress_level_9[:5])

### stress 

In [15]:
import time
def build_features(feats, stress_labels, student_id=2):
    t0 = time.time()
    location = feats['gps_details']
    location = index_time_to_df(location)
    # 1,2,3 - basic stats
    distance = to_distance(location)
    stress_labels = get_feat_distance_covered(distance, stress_labels)
    stress_labels = get_std_distance(distance, stress_labels)
    
    # 4 - unique tiles
    areas = build_spatial_map(location) # time -> tile
    stress_labels = bin_areas_by_day(areas, stress_labels)
    stress_labels = unique_tile_feats(stress_labels)
    
    # 5 - convex
    stress_labels = convex_hull_feats(location, stress_labels)
    
    # 6 - edit-disatance tiles
    stress_labels = prev_edit_dist_feats(stress_labels)

    # 7 - cluster
    stress_labels = location_cluster_feats(location, stress_labels)
    
    # 8 - entropy
    stress_labels = entropy_feats(distance, stress_labels)
    
    # 9,10,11,12 - temporal
    stress_labels = temporal_feats(stress_labels)
    
    feature_set = stress_labels
    print('student {} features built: {} sec'.format(student_id, time.time()-t0))
#     feature_set = pd.concat([total_distance, max_displacement, std_distances, unique_tiles, tile_changes, 
#                              convex_areas, location_cluster, entropies, temporal_data], axis=1, sort=False)
    
    features_list = ['distance', 'displacement', 'distance_std', 'unique_tile_count', 'convex_hull_area',
                    'tile_change','cluster', 'entropy', 'start_term', 'end_term', 'mid_term', 'weekends', 'stress'] 
    feature_set = pd.DataFrame(feature_set[features_list])
    feature_set = feature_set.loc[:,~feature_set.columns.duplicated()]
    return feature_set
    
# build_features(feats, stress_labels, student_id=43)

In [27]:
# labels - stress
def build_dataset(student_id=43):
    feats = load_features(student_id=student_id)
    stress_labels = get_stress_labels_with_time(feats)
    dataset = build_features(feats, stress_labels, student_id = student_id)
#     dataset = pd.concat([features, stress_labels], axis=1, join='inner', sort=False)
    dataset['student_id'] = [student_id for i in range(len(dataset))]
    dataset = dataset.reset_index()
    dataset = dataset.fillna(0.0)
    return dataset



In [28]:
STUDENTS_LIST = [1,2, 4 ,7,10, 22, 24, 33, 42, 46, 49, 53, 57]
STUDENT_LIST = list(range(1, ))
datasets = []
for sid in STUDENTS_LIST:
    dataset = build_dataset(student_id=sid)
    print("student {} data-length: {}".format(sid, len(dataset)))
    datasets.append(dataset)


student 1 loaded feat: stress_details.csv
student 1 loaded feat: gps_details.csv
student 1 features built: 8.12446403503418 sec
student 1 data-length: 36
student 2 loaded feat: stress_details.csv
student 2 loaded feat: gps_details.csv
student 2 features built: 8.806098937988281 sec
student 2 data-length: 35
student 4 loaded feat: stress_details.csv
student 4 loaded feat: gps_details.csv
student 4 features built: 8.356294870376587 sec
student 4 data-length: 46
student 7 loaded feat: stress_details.csv
student 7 loaded feat: gps_details.csv
student 7 features built: 4.78757905960083 sec
student 7 data-length: 58
student 10 loaded feat: stress_details.csv
student 10 loaded feat: gps_details.csv
student 10 features built: 14.440972089767456 sec
student 10 data-length: 112
student 22 loaded feat: stress_details.csv
student 22 loaded feat: gps_details.csv
student 22 features built: 10.255989074707031 sec
student 22 data-length: 60
student 24 loaded feat: stress_details.csv
student 24 loaded 

In [29]:
all_dataset = pd.concat(datasets, axis=0, sort=False)
all_dataset = all_dataset.sample(frac=1)
display(len(all_dataset))
# display(all_dataset)

710

In [30]:
data_path = module_path + "/data"
file_path = data_path + "/gatis_all.pkl"
print(file_path)
print(os.path.exists(data_path))
all_dataset.to_pickle(file_path)

/Users/nsimsiri/Documents/code/ml/MultiRes/student_life/data/gatis.pkl
True


In [31]:
mm = pd.read_pickle(file_path)
display(mm)

,time,distance,displacement,distance_std,unique_tile_count,convex_hull_area,tile_change,cluster,entropy,start_term,end_term,mid_term,weekends,stress,student_id
91,2013-04-25 16:34:14,2.761585,0.547704,0.108633,1.0,1055.675899,0.0,6.0,2.602271,0.0,0.0,1.0,0.0,0.0,10
43,2013-05-15 20:00:39,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,1.0,0.0,0.0,2.0,24
47,2013-04-16 08:00:08,1.740902,0.268766,0.054972,1.0,98.248766,2.0,4.0,2.952022,0.0,0.0,1.0,0.0,1.0,22
62,2013-05-30 02:20:26,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,1.0,0.0,0.0,2.0,33
77,2013-04-22 03:17:32,3.559978,0.561225,0.120343,1.0,403.916918,0.0,5.0,2.763989,0.0,0.0,1.0,0.0,0.0,10
10,2013-03-29 00:39:16,12.399209,6.045951,1.028009,1.0,149.229570,53.0,2.0,0.891268,1.0,0.0,0.0,0.0,2.0,24
23,2013-04-03 00:01:23,4.054641,0.540634,0.142673,1.0,1030.612086,0.0,6.0,2.566094,1.0,0.0,0.0,0.0,0.0,22
17,2013-04-05 06:56:11,12.691562,0.852496,0.293146,1.0,1618.768900,20.0,5.0,2.719588,1.0,0.0,0.0,0.0,1.0,46
8,2013-03-28 07:20:12,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,1.0,0.0,0.0,0.0,0.0,22
74,2013-04-21 01:23:55,2.475733,0.559131,0.107018,1.0,570.079265,1.0,5.0,2.484412,0.0,0.0,1.0,1.0,0.0,10


In [21]:
float is float('NaN')

False

In [37]:
import torch
torch.__version__
torch

'1.0.1.post2'